### Define workspace

define workspace and variables

In [1]:
import arcpy
arcpy.env.overwriteOutput = True

# authentication to access secured data
from arcgis.gis import GIS
from arcgis.features import FeatureLayer

gis = GIS('pro')
db = arcpy.env.workspace = r"C:\Users\AhjungKim\Documents\ArcGIS\Projects\SunCloud_TDM_Conflation\tdm_1221.gdb"
routes_url = 'https://services6.arcgis.com/clPWQMwZfdWn4MQZ/arcgis/rest/services/Sun_Cloud_Routes/FeatureServer/2'
tdm = 'SunCloudTDMnet2019_2050Volumes'

resuable functions

In [19]:
def save_fl(db, url, outname):
    fl = FeatureLayer(url)
    featureset = fl.query()
    featureset.save(db, outname)

export routes if it doesn't exist

In [20]:
routes = 'sun_cloud_routes'

if arcpy.Exists(routes) == False:
    save_fl(db, routes_url, routes)
else:
    print('sun cloud routes exists')
    

sun cloud routes exists


project if route's SR not in wkid = 2223

In [21]:
spatial_ref = arcpy.Describe('sun_cloud_routes').spatialReference
projected = 'sun_cloud_routes_project'
if spatial_ref.factoryCode != '2223' and arcpy.Exists(projected) == False:
    print('anim')
    arcpy.management.Project('sun_cloud_routes', 
     projected,
    'PROJCS["NAD_1983_StatePlane_Arizona_Central_FIPS_0202_Feet_Intl",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",700000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-111.9166666666667],PARAMETER["Scale_Factor",0.9999],PARAMETER["Latitude_Of_Origin",31.0],UNIT["Foot",0.3048]]', "WGS_1984_(ITRF00)_To_NAD_1983", 'PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]', 
    'PRESERVE_SHAPE')
else:
    print('in correct SR')

in correct SR


project the tdm layer 

In [22]:
#TODO: make this a function
spatial_ref = arcpy.Describe(tdm).spatialReference
projected = 'tdm_project'
target_sr = arcpy.Describe('sun_cloud_routes_project').spatialReference
if spatial_ref.factoryCode != '2223' and arcpy.Exists(projected) == False:
    arcpy.management.Project(tdm, projected, target_sr)
    print('project completed')
else:
    print('in correct SR')

in correct SR


unsplit the source if attributes are identical

In [23]:
arcpy.management.UnsplitLine("tdm_project", "tdm_project_unsplit", "ST_NAME;ABDAYVOL2019;BADAYVOL2019;TOTDAYVOL2019;ABDAYVOL2050;BADAYVOL2050;TOTDAYVOL2050")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\SunCloud_TDM_Conflation\\tdm_1221.gdb\\tdm_project_unsplit'>

create end points from the source tdm.  we'll use this to split the destination

In [24]:
arcpy.management.FeatureVerticesToPoints("tdm_project_unsplit", "tdm_end_pts", "BOTH_ENDS")


<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\SunCloud_TDM_Conflation\\tdm_1221.gdb\\tdm_end_pts'>

split the routes at tdm end points (don't use the snap tool)

In [25]:
arcpy.management.SplitLineAtPoint("sun_cloud_routes_project", "tdm_end_pts", "routes_unsplit_endpt", "40 Meters")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\SunCloud_TDM_Conflation\\tdm_1221.gdb\\routes_unsplit_endpt'>

create points along the tmd line

In [26]:
arcpy.management.GeneratePointsAlongLines('tdm_project_unsplit', 'tdm_points_8m', "DISTANCE", "8 Meters", None, "NO_END_POINTS")
arcpy.FeatureClassToFeatureClass_conversion("tdm_points_8m", 
                                            db, 
                                            "tdm_points_8m_backup")

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\SunCloud_TDM_Conflation\\tdm_1221.gdb\\tdm_points_8m_backup'>

get the source fields to transfer.

In [27]:
def containsNumber(value):
    for character in value:
        if character.isdigit():
            return True
    return False

In [28]:

fields = arcpy.ListFields('tdm_project_unsplit')
# for field in fields:
#     print("{0} is a type of {1} with a length of {2}"
#           .format(field.name, field.type, field.length))

# fields to conflate
def convert_type(f):
    if f.upper()=='INTEGER':
        return 'LONG'
    elif f.upper()=='STRING':
        return 'TEXT'
    else:
        return f


fields = [[field.name, convert_type(field.type)] for field in fields if field.required is False and field.name.upper() not in ('ST_NAME')]
fields


[['ABDAYVOL2019', 'LONG'],
 ['BADAYVOL2019', 'LONG'],
 ['TOTDAYVOL2019', 'LONG'],
 ['ABDAYVOL2050', 'LONG'],
 ['BADAYVOL2050', 'LONG'],
 ['TOTDAYVOL2050', 'LONG']]

add new fields to destination

In [29]:
arcpy.management.AddFields('routes_unsplit_endpt', fields)

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\SunCloud_TDM_Conflation\\tdm_1221.gdb\\routes_unsplit_endpt'>

verify new fields have been added

In [30]:
fields = arcpy.ListFields('routes_unsplit_endpt')
for field in fields:
    print(field.name)

OBJECTID
Shape
route_id
functional_class
ORIG_FID
ORIG_SEQ
Shape_Length
ABDAYVOL2019
BADAYVOL2019
TOTDAYVOL2019
ABDAYVOL2050
BADAYVOL2050
TOTDAYVOL2050


snap points to the source 

In [33]:
# field_names = [item[0] for item in fields]
# fields_to_update = field_names

# field_names.insert(0, 'SHAPE@')
# field_names.insert(1,'route_id')
# field_names.insert(2,'OID@')
# st_index = field_names.index('st_name')


# filter snapped points via fuzzy string match
import re
def strip_stop_words(x):
    # remove leading zeros
    x =  x.lstrip("0")

    stop_words = ['avenue', 'ave', 
                  'boulevard', 'blvd', 
                  'drive', 'dr', 
                  'freeway', 'frwy', 'fwy',
                  'lane', 'ln',
                  'parkway', 'pkwy',  
                  'road', 'rd', 
                  'route', 'rte',
                  'street', 'st', 
                  'trail', 'tr',
                  'way',
                  'ditch', 'canal', 'wash', 'channel', 'aqueduct', 'creek', 'river', 'adot', 'maintenance',
                  'railroad', 'chn', 'drainage', 'lake', 'siphons', 'track', 'bnsf', 
                  'place', 'pl', 
                  'l', 'n', 's', 'e', 'w', 'irr', 'i', 'us', 'to', '-', 'from',
                  'sl', 'sr', 'loop', 'lp', 'frtg', 
                  'sb', 'wb', 'eb', 'nb', 'direct', 'hov', 'ramp', 'n-w', 'w-s', 'n-e']
    x = re.split(r'\'|\s|;|,|/|-|\(|\)', x)
    # print(x)
    x = [y for y in x if y.lower() not in stop_words]
    # clean_list = [n for n in x if n.strip()]
    remove_zero = [item.lstrip('0') for item in x]
    clean= [n for n in remove_zero if n.strip()]
    return(clean)

###  Recursively snap and attribute (TODO: make this a function)

In [2]:
# snap by 20 meters

vertices = 'tdm_points_8m'
destination = 'routes_unsplit_endpt'
snap_config =  [[destination, "EDGE", "1 Meters"]]
arcpy.edit.Snap(vertices, snap_config)

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\SunCloud_TDM_Conflation\\tdm_1221.gdb\\tdm_points_8m'>

In [46]:
fields = arcpy.ListFields('routes_unsplit_endpt')
for field in fields:
    print(field.name)

OBJECTID
Shape
route_id
functional_class
ORIG_FID
ORIG_SEQ
Shape_Length
ABDAYVOL2019
BADAYVOL2019
TOTDAYVOL2019
ABDAYVOL2050
BADAYVOL2050
TOTDAYVOL2050
notes


In [40]:
arcpy.AddField_management('routes_unsplit_endpt', 'notes', 'TEXT')

<Result 'C:\\Users\\AhjungKim\\Documents\\ArcGIS\\Projects\\SunCloud_TDM_Conflation\\tdm_1221.gdb\\routes_unsplit_endpt'>

In [5]:
from distutils.command.clean import clean
import statistics

point = 'tdm_points_8m'
line = 'routes_unsplit_endpt'

arcpy.Delete_management('pt_layer')
arcpy.Delete_management('line_layer')
arcpy.management.MakeFeatureLayer(point, 'pt_layer')
# 'TOTDAYVOL2019 is null'
arcpy.management.MakeFeatureLayer(line, 'line_layer', where_clause='TOTDAYVOL2019 is null')
# 'OBJECTID = 7019'

route_selection = arcpy.management.SelectLayerByLocation('line_layer', 'INTERSECT', 'pt_layer')

cnt = int(arcpy.GetCount_management(route_selection).getOutput(0))
print(cnt, 'route segments intersect with points')

if cnt>0:
    route_fields = ['SHAPE@', 'OID@', 'ABDAYVOL2019', 'BADAYVOL2019', 'TOTDAYVOL2019', 'ABDAYVOL2050', 'BADAYVOL2050', 'TOTDAYVOL2050', 'notes']
    with arcpy.da.UpdateCursor(route_selection, route_fields, "notes is null") as cursor:
        for row in cursor:
            if row[0]:
                line_geom = row[0]
                oid = row[1]
                print("oid", oid)
                selection = arcpy.management.SelectLayerByLocation('pt_layer', 'INTERSECT', line_geom)
                
                cnt = int(arcpy.GetCount_management(selection).getOutput(0))
                print(cnt, 'overlaps')
                
                # proceed if more than one coinciding points
                if cnt>1:

                    if(1==1):
                        mode_oid = statistics.multimode([row for row in arcpy.da.SearchCursor(selection, ['ORIG_FID', 'ST_NAME'])])
                        st_name = (mode_oid[0])[1]
                        print(st_name)

                        if len(mode_oid) == 1:
                            if not st_name:

                                # print('tdm:', mode_oid)
                                where = 'ORIG_FID={0}'.format(mode_oid[0][0])
                                print(where)
                                subset = arcpy.management.SelectLayerByAttribute('pt_layer', 'SUBSET_SELECTION', where_clause=where)
                                # TODO: make this a function, use list index, check list has a single unique value
                                row[2] = list(set([row[0] for row in arcpy.da.SearchCursor(subset, ['ABDAYVOL2019'])]))[0]
                                row[3] = list(set([row[0] for row in arcpy.da.SearchCursor(subset, ['BADAYVOL2019'])]))[0]
                                row[4] = list(set([row[0] for row in arcpy.da.SearchCursor(subset, ['TOTDAYVOL2019'])]))[0]
                                row[5] = list(set([row[0] for row in arcpy.da.SearchCursor(subset, ['ABDAYVOL2050'])]))[0]
                                row[6] = list(set([row[0] for row in arcpy.da.SearchCursor(subset, ['BADAYVOL2050'])]))[0]
                                row[7] = list(set([row[0] for row in arcpy.da.SearchCursor(subset, ['TOTDAYVOL2050'])]))[0]
                            else:
                                print("no st name")
                                row[8] = 'no st_name'    
                        else:
                            row[8] = 'multimode'    

                        # print(row)
                        cursor.updateRow(row)
                # else:
                #     print('no overlap')
                #     row[8] = 'no overlap'
                #     cursor.updateRow(row)
else:
    print("null line doesn't overlap with points")



13036 route segments intersect with points
oid 2
3 overlaps
I-8
no st name
oid 3
112 overlaps
I-8
no st name
oid 4
48 overlaps
I-8
no st name
oid 5
2 overlaps
 
oid 8
4 overlaps
I-8
no st name
oid 9
2 overlaps
I-8
no st name
oid 10
389 overlaps
I-8
no st name
oid 11
30 overlaps
I-8
no st name
oid 12
1 overlaps
oid 13
1 overlaps
oid 14
2 overlaps
 
oid 15


In [3]:
# delete the used points so they don't participate in snapping again.
arcpy.Delete_management('pt_layer')
arcpy.Delete_management('line_layer')
arcpy.management.MakeFeatureLayer(point, 'pt_layer')
arcpy.management.MakeFeatureLayer(line, 'line_layer', 'ABDAYVOL2019 is not null')

pt_selection = arcpy.management.SelectLayerByLocation('pt_layer', 'INTERSECT', 'line_layer')

cnt = int(arcpy.GetCount_management(pt_selection).getOutput(0))
print(cnt)
if cnt>0 :
    arcpy.DeleteFeatures_management(pt_selection)


714164
